                              Физический факультет МГУ им.М.В.Ломоносова
                          Кафедра "Физико-математических методов управления"
                              
                              
                              
                              
                              
                          Курсовая работа по предмету "Численные методы в физике"
                "Решение уравнения колебаний методом Рунге-Кутты 4-ого порядка точности"
                              
                              
                              
                              
                              
                          Работу выполнил студент 442 группы Костин Никита

Уравнение колебаний математического маятника с вынуждающей силой и затуханием выглядит следующим образом: 

$$ y"(t)+2 \gamma y'(t)+w_0^2 y(t)=g(t) \tag{1}$$

Начальные условия для этой задачи задаются следующим образом:

$$ y(0)=y_0, y'(0)=y'_0 $$

Представим уравнение (1) второго порядка в виде системы уравнений первого порядка:

$$ 
\begin{cases}
    y'_1(t)=y_2(t) \\
    y'_2(t)=g(t)-2 \gamma y_2(t)-w_0^2 y_1(t)
\end{cases} \tag{2}
$$
Составим векторное уравнение из системы (2), обозначив 
$$ \vec{y} = 
\begin{equation*} 
    \left(
    \begin{array}{cc} 
    y_1 \\ 
    y_2 
    \end{array}
    \right)
\end{equation*} $$

Тогда получим векторную запись системы (2):
$$
\begin{equation*}
\left(
\begin{array}{ccccc}
y'_1(t) \\
y'_2(t)
\end{array}
\right)
= \vec{F}(t, \vec{y}) =
\left(
\begin{array}{ccc}
  y_2 \\
  g(t) - 2 \gamma y_2 - w_0^2 y_1
\end{array}
\right)
\end{equation*} \tag{3}
$$

Теперь построим равномерную сетку с шагом h и количеством точек. равным epoch.

$$ t_n = n h,    n=0,\ldots,epoch $$

Значения искомой вектор-функции обозначим на сетке как $$ \vec{y}_n = \vec{y}(t_n) $$

Суть метода Рунге-Кутты 4-ого порядка состоит в итерационном вычислении четырех поправок $k_1, k_2, k_3, k_4$ по формулам: $$ \vec{k_1}=\vec{f}(t_n,\vec{y_n})$$ 

$$\vec{k_2}=\vec{f}(t_n+\frac{h}{2},\vec{y_n}+\frac{h}{2} \vec{k_1})$$

$$\vec{k_3}=\vec{f}(t_n+\frac{h}{2},\vec{y_n}+\frac{h}{2} \vec{k_2})$$

$$\vec{k_4}=\vec{f}(t_n+h,\vec{y_n}+h \vec{k_3})$$, где h - шаг сетки.

Следующее значение функции на сетке равно:

$$\vec{y}_{n+1} = \vec{y_n} + \frac{h}{6} (\vec{k_1} + 2 \vec{k_2} + 2 \vec{k_3} + \vec{k_4}) $$

Возьмем $f(t)$ равной $A sin(wt)$. Напишем алгоритм Рунге-Кутты 4-ого порядка, ниже приведен код и графики.

In [ ]:
# Код программы
%matplotlib inline
import cmath
import numpy as np
import matplotlib.pyplot as plt

# Вынуждающая сила
def g(t):
    A = 1
    w = 100
    return A * cmath.sin(w * t)

# Класс векторов
class dq:
    def __init__(self, y_0, y_dot):
        self.y_0 = y_0
        self.y_dot = y_dot
        self.h = 0.01
        self.t = 0
        self.w_0_squared = 1
        self.gamma = 0.01
        self.y = np.array([y_0, y_dot])
        self.F = np.array([self.y[1],
                           (g(self.t) - 2*self.gamma*self.y[1] - self.w_0_squared*self.y[0])])

    def F_vec(self, t, y):
        self.F = np.array([y[1], (g(t) - 2*self.gamma*y[1] - self.w_0_squared*y[0])])
        return self.F

    def k1(self, t):
        k1 = self.F_vec(t, self.y)
        return k1

    def k2(self, k1, t):
        k2 = self.F_vec(t + self.h/2, self.y + self.h/2*k1)
        return k2

    def k3(self, k2, t):
        k3 = self.F_vec(t + self.h/2, self.y + self.h/2*k2)
        return k3

    def k4(self, k3, t):
        k4 = self.F_vec(t + self.h, self.y + self.h*k3)
        return k4

    def solution(self, t):
        A = 1
        w = 100
        s = cmath.sqrt(self.gamma*self.gamma-self.w_0_squared)
        c = self.w_0_squared*self.w_0_squared-w*w
        c1 = (self.y_0*(s-self.gamma)-self.y_dot)/(2*s)
        c2 = (self.y_dot+self.y_0*(s+self.gamma))/(2*s)
        return c1*cmath.exp(-t*(s+self.gamma)) + c2*cmath.exp(t*(s-self.gamma)) + \
               A*(cmath.sin(w*t)*c + cmath.cos(w*t)*2*self.gamma*w)/(c*c + 4*self.gamma*self.gamma*w*w)

# Цикл алгоритма Рунге-Кутты
epoch = 4000
Y = dq(5, -1)
func = np.array([0 for _ in range(epoch)])
func_speed = np.array([0 for _ in range(epoch)])
coords = np.array([i*Y.h for i in range(epoch)])
sol = np.array([Y.solution(coords[i]) for i in range(epoch)])
error = np.array([0 for _ in range(epoch)])
func[0] = Y.y[0]
error[0] = abs(sol[0]-func[0])
for i in range(1, epoch):
    K1 = Y.k1(coords[i-1])
    K2 = Y.k2(K1, coords[i-1])
    K3 = Y.k3(K2, coords[i-1])
    K4 = Y.k4(K3, coords[i-1])
    Y.y = Y.y + (K1 + 2*K2 + 2*K3 + K4)*Y.h/6
    func[i] = Y.y[0]
    func_speed[i] = Y.y[1]
    error[i] = sol[i] - func[i]

# Построение графиков
plt.title("График зависимости координаты маятника от времени")
plt.plot(coords, func, 'r-', label="Численный расчёт")
plt.plot(coords, sol, 'b-', label="Аналитический расчёт")
plt.axis([0, 40, -5, 7])
plt.grid()
plt.text(0.2, 6.5, r"$\gamma = 0.01, w_0 = 1, y_0=5, y'_0=-1$ ", fontsize=11)
plt.text(0.2, 5.5, r"$A = 1, w = 100, h=0.01$ ", fontsize=11)
plt.xlabel("Время, с")
plt.ylabel("Координата, м")
plt.legend()
#plt.plot(coords, error, 'b-')
plt.savefig('graph1.png')
plt.show()
